## BLERSSI Training & Deployment in Hybrid environment ( Cisco UCS & Azure)

BLERSSI model training on Cisco UCS server & model deployment on Azure Cloud.

### Clone Cisco Kubeflow Starter Pack repository

In [ ]:
BRANCH_NAME="hybrid" #Provide git branch as "master"/"dev"/"hybrid"
! git clone -b $BRANCH_NAME https://github.com/CiscoAI/cisco-kubeflow-starter-pack.git  

## Install required libraries

In [ ]:
!pip install kfp azureml-core --user

## Restart notebook kernel

In [ ]:
from IPython.display import display_html
display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)

## Import required libraries

In [ ]:
import kfp
import os
import calendar
import time
from kubernetes import client
import json
import numpy as np
import base64
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.authentication import ServicePrincipalAuthentication
from azureml.core import Workspace

## Component file declarations

In [ ]:
path='cisco-kubeflow-starter-pack/apps/networking/ble-localization/hybrid/azure/pipelines/'
component_root_train = path + 'components/train_model/'
component_root_register = path + 'components/register_model/'
component_root_deploy = path + 'components/deploy_model/'

## Load components from respective .yaml files

In [ ]:
azure_train_op = kfp.components.load_component_from_file(os.path.join(component_root_train, 'component.yaml'))
azure_register_op = kfp.components.load_component_from_file(os.path.join(component_root_register, 'component.yaml'))
azure_deploy_op = kfp.components.load_component_from_file(os.path.join(component_root_deploy, 'component.yaml'))

## Define volume & volume mounts

In [ ]:
nfs_pvc = client.V1PersistentVolumeClaimVolumeSource(claim_name='nfs')
nfs_volume = client.V1Volume(name='nfs', persistent_volume_claim=nfs_pvc)
nfs_volume_mount = client.V1VolumeMount(mount_path='/mnt/', name='nfs')

## Declare environment variables for Azure Credentials

In [ ]:
workspace_name = os.getenv('WORKSPACE_NAME')
subscription_id = os.getenv('SUBSCRIPTION_ID')
resource_group = os.getenv('RESOURCE_GROUP')
tenant_id = os.getenv('TENANT_ID')
service_principal_id = os.getenv('SERVICE_PRINCIPAL_ID')
service_principal_password = os.getenv('SERVICE_PRINCIPAL_PASSWORD')

## Create timestamp

In [ ]:
timestamp = str(calendar.timegm(time.gmtime()))
print(timestamp)
model = "testmodel"+timestamp
service = "testservice"+timestamp

## Define pipeline function

In [ ]:
def azure_pipeline():
    
    #Define task for training BLERSSI data
    azure_train_task = azure_train_op()
    azure_train_task.add_volume(nfs_volume)
    azure_train_task.add_volume_mount(nfs_volume_mount)
    
    #Define task for registering BLERSSI model on Azure
    azure_register_task = azure_register_op(workspace_name=workspace_name,
                                           subscription_id=subscription_id,
                                           resource_group=resource_group,
                                           model_name=model,
                                           tenant_id=tenant_id,
                                           service_principal_id=service_principal_id,
                                           service_principal_password=service_principal_password)
    
    azure_register_task.add_volume(nfs_volume)
    azure_register_task.add_volume_mount(nfs_volume_mount)
    azure_register_task.after(azure_train_task)
    
    #Define Task for deploying BLERSSI model on Azure 
    azure_deploy_task = azure_deploy_op(workspace_name=workspace_name,
                                        subscription_id=subscription_id,
                                        resource_group=resource_group,
                                        model_name=model,
                                        service_name=service,
                                        tenant_id=tenant_id,
                                        service_principal_id=service_principal_id,
                                        service_principal_password=service_principal_password)

    azure_deploy_task.after(azure_register_task)

## Create & execute Azure deployment pipeline

In [ ]:
#Create a pipeline run
kfp.Client().create_run_from_pipeline_func(azure_pipeline, arguments={})

## Authenticate Azure account

In [ ]:
svc_pr_password = os.environ.get("AZUREML_PASSWORD")
svc_pr = ServicePrincipalAuthentication(
tenant_id=tenant_id,
service_principal_id=service_principal_id,
service_principal_password=service_principal_password)

ws = Workspace(
subscription_id=subscription_id,
resource_group=resource_group,
workspace_name=workspace_name,
auth=svc_pr
)

## Use client to retrieve Azure scoring URI

In [ ]:
service = Webservice(workspace=ws, name=service)
print(service.scoring_uri)

## View logs of Azure scoring URI creation

In [ ]:
service.get_logs()

## Process input data for prediction input

In [ ]:
X=[[-200,-200,-200,-200,-200,-63,-200,-200,-200,-200,-200,-200,-200]]
test_samples = json.dumps({"data": X})
test_samples = bytes(test_samples, encoding='utf8')

## Predict using Azure scoring URI

In [ ]:
service.run(input_data=test_samples)